# Masking And Trying CNN On It

In this file i tried something new for me . I have created masks for the dogs and cats images using threshold and then trained the model on it and done some predictions . 

# Importing the packages

In [ ]:
import pandas as pd
import numpy as np
import cv2
import keras.models as M
import keras.layers as L
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import clear_output
import keras

# Having a look at the data

In [ ]:
image=Image.open('../input/dogs-cats-images/dataset/training_set/dogs/dog.2.jpg')
image=image.convert('L')
image=np.array(image)
plt.imshow(image)

# Having a look at the histogram to get the threshold of the image 

In [ ]:
a,b,x=plt.hist(image.flatten())


### Threshold looks like it lies between 135-160

# Let's Make a Function to get the threshold and make masked images

In [ ]:
def get_threshold(image) :
    a,b,x=plt.hist(image.flatten())
    arg=np.argmax(a)
    threshold=[b[arg],b[arg+1]]
    plt.clf()
    return threshold
def make_mask(path,color='blue',print_plot=False):
    image=Image.open(path)
    image=image.resize((150,150))
    image=image.convert('L')
    image=np.array(image)
    threshold=get_threshold(image)
    if color=='blue' :
        n=[0,0,255]
    masked_image=[]
    for i in image.flatten() :
        if i >= threshold[0] and i<=threshold[1] :
            masked_image.append(255)
        else:
            masked_image.append(0)
    masked_image=np.array(masked_image)
    masked_image=masked_image.reshape(image.shape)
    if print_plot==True :
        plt.imshow(masked_image)
    return masked_image
        
    

In [ ]:
arr=make_mask('../input/dogs-cats-images/dataset/training_set/dogs/dog.1001.jpg',print_plot=True)

# Making Masked Data (Only 1000 for each class since it takes some time)

In [ ]:
# Adding the paths
dog_path='../input/dogs-cats-images/dataset/training_set/dogs'
cats_path='../input/dogs-cats-images/dataset/training_set/cats'

In [ ]:
# Making the data
X=[]
y=[]
count=0
for i in os.listdir(dog_path)[:1000]:
    print(count*100/1000)
    clear_output(wait=True)
    count+=1
    X.append(make_mask(dog_path+'/'+i))

y=[1]*1000

In [ ]:
# Making mask image data for cats
x2=[]
y2=[]
count=0
for i in os.listdir(cats_path)[:1000]:
    print(count*100/1000)
    clear_output(wait=True)
    count+=1
    x2.append(make_mask(cats_path+'/'+i))
y2=[0]*1000

In [ ]:
# Getting all the data together
X.extend(x2)
y.extend(y2)
X=np.array(X)
y=np.array(y)
X2=X/255

# Doing Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X2, y, test_size=0.15, random_state=42 )

# Making the model

In [ ]:
model=M.Sequential()
model.add(L.Reshape((150,150,1)))
model.add(L.Conv2D(kernel_size=(3,3),filters=32,input_shape=(150,150),activation='relu'))
model.add(L.MaxPool2D(pool_size=(2,2)))
model.add(L.Conv2D(kernel_size=(3,3),filters=64,activation='relu'))
model.add(L.MaxPool2D(pool_size=(2,2)))
model.add(L.Flatten())
model.add(L.Dense(1000,activation='tanh'))
model.add(L.Dense(100,'relu'))
model.add(L.Dense(2,activation='softmax'))

# Having a look at the data

In [ ]:
model.summary()

# Compiling the model

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Training The model

In [ ]:
model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

# Saving the model

In [ ]:
keras.models.save_model(model,'mask_model.h5')

# Making a function to do some predicitons for us

In [ ]:
def make_predictions(path) :
    l=['Cats','Dogs']
    print('The masked image is :')
    arr=make_mask(path,print_plot=True)
    arr=np.array(arr)
    arr=arr.reshape(1,150,150)
    clas=np.argmax(model.predict(arr),axis=-1)
    print(l[clas[0]])
    

# Doing some predictions

In [ ]:

make_predictions('../input/dogs-cats-images/dataset/test_set/dogs/dog.4005.jpg')

In [ ]:
make_predictions('../input/dogs-cats-images/dataset/test_set/cats/cat.4008.jpg')

# I can't understand the second image at all but our model predicted it right :)

# Thank you :)